In [18]:
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import seaborn as sns

mental_impairment_data = pd.DataFrame([
    (1, 'Well', 1, 1),
    (2, 'Well', 1, 9),
    (3, 'Well', 1, 4),
    (4, 'Well', 1, 3),
    (5, 'Well', 0, 2),
    (6, 'Well', 1, 0),
    (7, 'Well', 0, 1),
    (8, 'Well', 1, 3),
    (9, 'Well', 1, 3),
    (10, 'Well', 1, 7),
    (11, 'Well', 0, 1),
    (12, 'Well', 0, 2),
    (13, 'Mild', 1, 5),
    (14, 'Mild', 0, 6),
    (15, 'Mild', 1, 3),
    (16, 'Mild', 0, 1),
    (17, 'Mild', 1, 8),
    (18, 'Mild', 1, 2),
    (19, 'Mild', 0, 5),
    (20, 'Mild', 1, 5),
    (21,' Mild', 1, 9),
    (22, 'Mild', 0, 3),
    (23, 'Mild', 1, 3),
    (24, 'Mild', 1, 1),
    (25, 'Moderate', 0, 0),
    (26, 'Moderate', 1, 4),
    (27, 'Moderate', 0, 3),
    (28, 'Moderate', 0, 9),
    (29, 'Moderate', 1, 6),
    (30, 'Moderate', 0, 4),
    (31, 'Moderate', 0, 3),
    (32, 'Impaired', 1, 8),
    (33, 'Impaired', 1, 2),
    (34, 'Impaired', 1, 7),
    (35, 'Impaired', 0, 5),
    (36, 'Impaired', 0, 4),
    (37, 'Impaired', 0, 4),
    (38, 'Impaired', 1, 8),
    (39, 'Impaired', 0, 8),
    (40, 'Impaired', 0, 9),
], columns=['Subject', 'Mental Impairment', 'SES', 'Life Events'])

mental_impairment_data = mental_impairment_data.set_index('Subject')
mental_impairment_data
mental_impairment_data

,Mental Impairment,SES,Life Events
Subject,,,
1,Well,1,1
2,Well,1,9
3,Well,1,4
4,Well,1,3
5,Well,0,2
6,Well,1,0
7,Well,0,1
8,Well,1,3
9,Well,1,3
